# 딥러닝 기초 6강(2018.09.15)

In [1]:
options(scipen=999)
getwd()

[1] "C:/Users/Playdata/OneDrive/R/LS_Deep"

1. 문제상황 이해
2. 데이터 이해
###EDA
3. 전처리
4. feature engineering

5. modeling

# 1. wine data로 LM, tree 실습

In [2]:
###1-a) data4_1
data4_1 <- read.csv("data4_1.csv", header = TRUE)
test_df <- data4_1
colnames(test_df)

[1] "fixed_acidity"        "volatile_acidity"     "citric_acid"         
 [4] "residual_sugar"       "chlorides"            "free_sulfur_dioxide" 
 [7] "total_sulfur_dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "type"

# a). test_set, train_set 나누기

In [3]:
set.seed(10)
wine <- test_df 

train_test_split <- sample(1:nrow(wine), 0.8*nrow(wine))

train_set <- wine[train_test_split, ]
test_set <- wine[-train_test_split, ]

write.csv(test_set, "test_set.csv", row.names=F)
write.csv(train_set, "train_set.csv", row.names=F)

rm(test_set)#remove test_set!!

# b) train_set만 가지고 학습하기

In [4]:
# 1.  train set will not be contaminated   
wine <- read.csv("train_set.csv")
wine[1,2]
# 2. split validation set 2:8 again
train_valid_split <- sample(1:nrow(wine), 0.8*nrow(wine))

[1] 0.26

# c) lm

In [5]:
real_quality <- wine$quality[train_valid_split]

model <- lm(quality~., data=wine[train_valid_split, ])

pred_quality_lm <- predict(model, wine[-train_valid_split, ])

validation_error_lm <- mean((real_quality - pred_quality_lm)^2)


Warning message in real_quality - pred_quality_lm:
"두 객체의 길이가 서로 배수관계에 있지 않습니다"

# d) tree

In [6]:
library(tree)

ERROR: Error in library(tree): there is no package called 'tree'


In [7]:
real_quality <- wine$quality[train_valid_split]

model <- tree(quality~., data=wine[train_valid_split, ])

pred_quality_tree <- predict(model, wine[-train_valid_split, ])

validation_error_tree <- mean((real_quality - pred_quality_tree)^2)

ERROR: Error in tree(quality ~ ., data = wine[train_valid_split, ]): 함수 "tree"를 찾을 수 없습니다


In [8]:
# e) lm과 tree의 validation_error(test_error) 비교

cat("validation_error_lm의 값: ", validation_error_lm)
cat("\n")
cat("validation_error_tree의 값: ", validation_error_tree)

validation_error_lm의 값:  1.010103


ERROR: Error in cat("validation_error_tree의 값: ", validation_error_tree): 객체 'validation_error_tree'를 찾을 수 없습니다


# 2. wine data로 GML 실습

## gml

generalized linear model(glm)
output이 따르는 확률 분포 정규분포

logistic regression은 ouput이 이항분포

In [9]:
library("glmnet")

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16



In [10]:
#  와인타입
table(wine$type)


   0    1 
1293 3904 

In [11]:
real_type <- wine$type[-train_valid_split]
  
model <- glm(formula =type~., 
             data = wine[train_valid_split, ],
             family ='binomial' )
  
pred_type <- predict(model, wine[-train_valid_split, ], type='response')

#확률 예측한것을 레이블롬 ㅏㄴ들자
pred_type_label <-ifelse(pred_type > 0.5, 1, 0) #true에 1, false에 0 


# accurcy 정확도
(251+780) / (251 + 6 + 3 +  780) accuracy: 정확도

In [12]:
table(real_type, pred_type_label)

         pred_type_label
real_type   0   1
        0 274   4
        1   3 759

# 여러 기준의 accuracy
### Recall, Precision

In [13]:
pred_type_label1 <-ifelse(pred_type > 0.5, 1, 0) #true에 1, false에 0 
pred_type_label2 <-ifelse(pred_type > 0.9, 1, 0) #true에 1, false에 0 
pred_type_label3 <-ifelse(pred_type > 0.2, 1, 0) #true에 1, false에 0  화이트 와인 기준 느슨하


table(real_type, pred_type_label1)
table(real_type, pred_type_label2)
table(real_type, pred_type_label3)


         pred_type_label1
real_type   0   1
        0 274   4
        1   3 759

         pred_type_label2
real_type   0   1
        0 275   3
        1  10 752

         pred_type_label3
real_type   0   1
        0 270   8
        1   1 761

# 3. Keraas CNN
https://towardsdatascience.com/how-to-implement-deep-learning-in-r-using-keras-and-tensorflow-82d135ae4889

784 -> 2(압축) -> 784 
loss를 mse로
원본 이미지를 넣어서, 원본 이미지를 예측하는 모델 

denoise autoencoder
<img src="https://www.doc.ic.ac.uk/~js4416/163/website/img/autoencoders/denoising-example.png" width="400">

convolutional autoencoder
<img src="https://cdn-images-1.medium.com/max/2000/1*8ixTe1VHLsmKB3AquWdxpQ.png" width="400">

In [14]:
#library(keras)
#install_keras()
# mnist <- dataset_mnist()

In [15]:
class(mnist)

ERROR: Error in eval(expr, envir, enclos): 객체 'mnist'를 찾을 수 없습니다


In [16]:
class(mnist)

ERROR: Error in eval(expr, envir, enclos): 객체 'mnist'를 찾을 수 없습니다


In [18]:
model <- keras_model_sequential() %>%
layer_dense(units = 2, input_shape = 784) %>%
layer_activtaion('relu') %>% 

layer_dense(units = 784) %>%
layer_activtaion('sigmoid') %>% 

compile(loss='mse', optimizer='adam')

fit(model, image_train, image_train)

ERROR: Error in keras_model_sequential() %>% layer_dense(units = 2, input_shape = 784) %>% : 함수 "%>%"를 찾을 수 없습니다


# 4. wine data로 XGboosting 실습
  
1. 파라미터

  1. eta: 작게 작으면 오버피팅 적음  
  2. max_depth : 나무 깊이, 오버피팅과 비례  
  3. subsample : 하나의 나무를 만들 때 일정 데이터만 샘플링하면 bagging 과 같은 효과가 나옴  
  4. lambda: ridge같은 것 l2 regularization  
  5. alpha: lasso 같은것 l1 regularization  

https://xgboost.readthedocs.io/en/latest/parameter.html


In [19]:
library(xgboost)

ERROR: Error in library(xgboost): there is no package called 'xgboost'


# a) train, validation, test set을 나눴음  

In [20]:
wine <- read.csv("train_set.csv")
set.seed(10)

train <- sample(1:nrow(wine), 0.8*nrow(wine))

input <- model.matrix(quality ~., data=wine)
output <- wine$quality

hyperparameter <- list(eta = 0.03,
                       max_depth = 8,
                       eta =0.8)

model <- xgboost(data = input[train, ],
                label = output[train],
                params = hyperparameter,
                nrounds = 30)

pred_output <- predict(model, input[-train,])
real_output <- output[-train]

val_error <- mean((real_output - pred_output)^2)

val_error

ERROR: Error in xgboost(data = input[train, ], label = output[train], params = hyperparameter, : 함수 "xgboost"를 찾을 수 없습니다


# b) 전체 데이터로 cross validataion

  ### xgb.cv : 하이퍼 파라미터 어떤 것이 중요한지 알 수 있음
  
  50번까지 뽑았는데, 이 때의 validation error(test-error)를 알 수 있음

In [21]:
wine_full <- read.csv("data4_1.csv")
set.seed(10)

train <- sample(1:nrow(wine_full), 0.8*nrow(wine_full))

input <- model.matrix(quality ~., data=wine_full)
output <- wine_full$quality

hyperparameter <- list(eta = 0.3,
                       max_depth = 8,
                       eta =0.8)

model <- xgb.cv(data = input[train, ],
                label = output[train],
                params = hyperparameter,
                nrounds = 50,
                nfold=5)

ERROR: Error in xgb.cv(data = input[train, ], label = output[train], params = hyperparameter, : 함수 "xgb.cv"를 찾을 수 없습니다


# c) 나무 갯수 99999로 하다가 earlystopping rounds 로 설정

validation error가 20번 똑같으면 멈추자



In [22]:
wine_full <- read.csv("data4_1.csv")
set.seed(10)

train <- sample(1:nrow(wine_full), 0.8*nrow(wine_full))

input <- model.matrix(quality ~., data=wine_full)
output <- wine_full$quality

hyperparameter <- list(eta = 0.03,
                       max_depth = 8,
                       eta =0.8)

model <- xgb.cv(data = input[train, ],
                label = output[train],
                params = hyperparameter,
                nrounds = 999999,
                nfold=5,
                early_stopping_rounds = 20)


ERROR: Error in xgb.cv(data = input[train, ], label = output[train], params = hyperparameter, : 함수 "xgb.cv"를 찾을 수 없습니다


***
Model_final <- xgboost(data=input,  
> label = output,   
> params = "__적절한 하이퍼파라미터__",  
>  nrounds = "__적절한 개수__")    

xgb.cv는 validation error를 계산하느 ㄴ것, 모델을 만드는 것이 아님
하이퍼 파라미터가 무엇이 중요한지 알 수 있음 

그리고 xgboost로 모델을 만들고, 평가함  


# 4. san 데이터로 random tree 실습

In [23]:
setwd("C:\\Users\\Playdata\\OneDrive\\R\\LS_Deep\\satander")
getwd()

[1] "C:/Users/Playdata/OneDrive/R/LS_Deep/satander"

In [24]:
library(ranger)
san  <- read.csv("train.csv")

In [25]:
#use 80% of random data as a training set
train <- sample(1:nrow(san), 0.8*nrow(san))

train_set <- san[train, ]
test_set <- san[-train, ]


write.csv(test_set, "test_set_san.csv", row.names=F)
write.csv(train_set, "train_set_san.csv", row.names=F)


In [26]:
# 새로 불러오기  
san <- read.csv("train_set_san.csv")

set.seed(10)

train <- sample(1:nrow(san), 0.8*nrow(san))
model <- ranger(target~.-ID, data=san[train, ], mtry = 2)

#predict에는 $prediction 해줘야함
pred_target <- predict(model, san[-train, ])$predictions
real_target <- san$target[-train]

val_error <- mean((real_target - pred_target)^2)
val_error

[1] 60439609203403

In [28]:
# 수개월 후 학습이 끝나면

test_set <- read.csv("test_set_san.csv")

model <- ranger(target~.-ID, data=san[train, ], mtry = 가장 적절한)

pred_target <- predict(mode, test_set)$predictions
real_target <- test_set$target

test_error <- mean((real_target - pred_target)^2)

test_error

ERROR: Error in parse(text = x, srcfile = src): <text>:5:61: 예상하지 못한 기호(symbol)입니다.
4: 
5: model <- ranger(target~.-ID, data=san[train, ], mtry = 가장 적절한
                                                               ^


*** 
# R^2 

0에 가까우면 안좋고, 1에 가까우면 좋다
머신러닝할ㄸ MSE 많이 씀, 

y_bar  trainng set의 평균